In [1]:
import tensorflow as tf  

In [2]:
import nltk
nltk.download()
                                                                                               

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import pandas as pd
import csv

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('faqData.json').read()
faq = json.loads(data_file)


In [6]:
from nltk import word_tokenize
import nltk
for intent in faq['faqData']:
    for pattern in intent['userMessage']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['label']))
        if intent['label'] not in classes:
            classes.append(intent['label'])

In [7]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))            
pickle.dump(classes,open('classes.pkl','wb'))

23 documents
5 classes ['doubt', 'goodbye', 'greeting', 'requirements_for_appointment', 'thanks']
57 unique lemmatized words ["'s", ',', 'a', 'age', 'am', 'anyone', 'appointment', 'are', 'awesome', 'bye', 'chatting', 'day', 'doctor', 'facing', 'for', 'free', 'good', 'goodbye', 'had', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'later', 'limit', 'me', 'minimum', 'need', 'next', 'nice', 'percentage', 'prerequisties', 'problem', 'proceed', 'query', 'required', 'see', 'test', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'what', 'with', 'you', 'your']


In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag axxxxxxxxxxxzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzznd '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\sanja\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

c:\Users\sanja\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
5/5 [==============================] - 1s 2ms/step - loss: 1.7194 - accuracy: 0.0435
Epoch 2/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7395 - accuracy: 0.1304
Epoch 3/200
5/5 [==============================] - 0s 4ms/step - loss: 1.5352 - accuracy: 0.2609
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.4285 - accuracy: 0.4348
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 1.4328 - accuracy: 0.4348
Epoch 6/200
5/5 [==============================] - 0s 4ms/step - loss: 1.2818 - accuracy: 0.6087
Epoch 7/200
5/5 [==============================] - 0s 2ms/step - loss: 1.4171 - accuracy: 0.4348
Epoch 8/200
5/5 [==============================] - 0s 4ms/step - loss: 1.2180 - accuracy: 0.6087
Epoch 9/200
5/5 [==============================] - 0s 2ms/step - loss: 1.1203 - accuracy: 0.5217
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0898 - accuracy: 0.6957
Epoch 11/200
5/5 [===========